In [18]:
import os
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

In [22]:
username = os.getenv('DB_USERNAME') # use username in your database connection 
password = os.getenv('DB_PASSWORD') # use password in your database connection 
hostname = "localhost"
port = "5432"
database_name = "medid"

In [20]:
# Create the engine
db_string = f"postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{database_name}"
engine = create_engine(db_string)

In [7]:
try:
    with engine.begin() as connection:
        connection.execute(text('DROP VIEW IF EXISTS JPD;'))
        connection.execute(text('DROP TABLE IF EXISTS d;'))
        connection.execute(text('DROP TABLE IF EXISTS p;'))
        connection.execute(text('DROP TABLE IF EXISTS s;'))
        connection.execute(text('DROP TABLE IF EXISTS t;'))
        connection.execute(text('DROP TABLE IF EXISTS u;'))
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

In [8]:
try:
    with engine.begin() as connection:
        # disease table
        connection.execute(text('''
            CREATE TABLE IF NOT EXISTS public.d
            (
                id INTEGER NOT NULL PRIMARY KEY,
                has_disease boolean,
                prob REAL NOT NULL
            )
        '''))
        connection.execute(text('''
                    INSERT INTO d(id, has_disease, prob)
                    VALUES (1, true, 0.1), (2, false, 0.9)
                '''))
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

In [9]:
try:
    with engine.begin() as connection:
        # disease table
        connection.execute(text('''
            CREATE TABLE IF NOT EXISTS public.p
            (
                id INTEGER NOT NULL PRIMARY KEY,
                d_id INTEGER,
                FOREIGN KEY (d_id) REFERENCES d(id),
                has_pathological_state boolean NOT NULL,
                prob REAL NOT NULL
            )
        '''))
        connection.execute(text('''
                    INSERT INTO p(id, d_id, has_pathological_state, prob)
                    VALUES (1, 1, true, 0.8), (2, 1, false, 0.2),
                     (3, 2, true, 0.15), (4, 2, false, 0.85)
                '''))
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

In [10]:
try:
    with engine.begin() as connection:
        # disease table
        connection.execute(text('''
            CREATE TABLE IF NOT EXISTS public.s
            (
                id INTEGER NOT NULL PRIMARY KEY,
                p_id INTEGER,
                FOREIGN KEY (p_id) REFERENCES p(id),
                has_symptom boolean NOT NULL,
                prob REAL NOT NULL
            )
        '''))
        connection.execute(text('''
                    INSERT INTO s(id, p_id, has_symptom, prob)
                    VALUES (1, 1, true, 0.7), (2, 1, false, 0.3),
                     (3, 2, true, 0.2), (4, 2, false, 0.8),
                     (5, 3, true, 0.7), (6, 3, false, 0.3),
                     (7, 4, true, 0.2), (8, 4, false, 0.8)
                '''))
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

In [13]:
try:
    with engine.begin() as connection:
        # action table
        connection.execute(text('''
            CREATE TABLE IF NOT EXISTS public.t
            (
                id INTEGER NOT NULL PRIMARY KEY,
                treat boolean NOT NULL
            )
        '''))
        connection.execute(text('''
                    INSERT INTO t(id, treat)
                    VALUES (1, TRUE), (2, FALSE)
                '''))
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

In [14]:
try:
    with engine.begin() as connection:
        # utility table
        connection.execute(text('''
            CREATE TABLE IF NOT EXISTS public.u
            (              
                t_id INTEGER NOT NULL,
                p_id INTEGER NOT NULL,
                d_id INTEGER NOT NULL,
                PRIMARY KEY (t_id, p_id, d_id),
                FOREIGN KEY (t_id) REFERENCES t(id),
                FOREIGN KEY (p_id) REFERENCES p(id),
                FOREIGN KEY (d_id) REFERENCES d(id),
                utility INTEGER NOT NULL
            )
        '''))
        connection.execute(text('''
                    INSERT INTO u(t_id, p_id, d_id, utility)
                    VALUES (1,1,1,10), (1,1,2,6), (1,2,1,8), (1,2,2,4),
                    (2,1,1,0), (2,1,2,2), (2,2,1,1), (2,2,2,10)
                '''))
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

The following code computes the joint probability distribution JPD as database view expressed as join query on all chance tables joined on the keys between the d, p, s tables.

In [15]:
try:
    with engine.begin() as connection:
        connection.execute(text('''
        CREATE OR REPLACE VIEW JPD
            AS
            SELECT d.has_disease, p.has_pathological_state, s.has_symptom, 
                d.prob * p.prob * s.prob AS prob
            FROM d
            JOIN p ON p.d_id = d.id
            JOIN s on s.p_id = p.id
        ''')) 
        print('Created view JPD.')
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

Created view JPD.


The following code computes all value tuples of the decision, symptom, and expected utility.

In [23]:
try:
    with engine.begin() as connection:
        result = connection.execute(text('''
            WITH tot AS (
                 SELECT has_symptom, SUM(prob) AS sum_prob FROM jpd GROUP BY has_symptom
            ),
            prob_data AS 
            (
                SELECT jpd.has_pathological_state, jpd.has_symptom, jpd.has_disease, 
                       jpd.prob / tot.sum_prob as prob1,
                       p.id as p_id, d.id as d_id
                FROM jpd
                JOIN tot ON jpd.has_symptom = tot.has_symptom
                JOIN d ON d.has_disease = jpd.has_disease
                JOIN p ON p.has_pathological_state = jpd.has_pathological_state
            )

            
            SELECT prob_data.has_symptom, t.treat, SUM(prob_data.prob1 * u.utility) as sum_utility 
            FROM prob_data
            JOIN u ON (u.p_id = prob_data.p_id AND u.d_id = prob_data.d_id)
            JOIN t ON (t.id = u.t_id)
            GROUP BY prob_data.has_symptom, t.id;
        '''))

    
    for row in result:
        print(row)

except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

(False, False, 8.977617133408785)
(False, True, 4.417328506708145)
(True, False, 5.603251768276095)
(True, True, 5.759349569678307)


The following code computes all value tuples for (symptom, decision) that maximize the expected utility, the probability of symptom occurrence, and the maximum expected utility resulting in each situation.

In [24]:
try:
    with engine.begin() as connection:
        result = connection.execute(text('''
        WITH tot AS (
     SELECT has_symptom, SUM(prob) AS sum_prob FROM jpd GROUP BY has_symptom
),
prob_data AS 
(
    SELECT jpd.has_pathological_state, jpd.has_symptom, jpd.has_disease, 
           jpd.prob / tot.sum_prob as prob1,
           p.id as p_id, d.id as d_id
    FROM jpd
    JOIN tot ON jpd.has_symptom = tot.has_symptom
    JOIN d ON d.has_disease = jpd.has_disease
    JOIN p ON p.has_pathological_state = jpd.has_pathological_state
),
sum_util_data AS 
(
    SELECT prob_data.has_symptom, t.treat, SUM(prob_data.prob1 * u.utility) as sum_utility 
    FROM prob_data
    JOIN u ON (u.p_id = prob_data.p_id AND u.d_id = prob_data.d_id)
    JOIN t ON (t.id = u.t_id)
    GROUP BY prob_data.has_symptom, t.id
),
max_utility_data AS
(
    SELECT *,
        RANK() OVER (PARTITION BY has_symptom ORDER BY sum_utility DESC) as rank
    FROM sum_util_data
)
SELECT max_utility_data.has_symptom, max_utility_data.treat, max_utility_data.sum_utility, tot.sum_prob, max_utility_data.sum_utility * tot.sum_prob as total_utility
FROM max_utility_data
JOIN tot ON max_utility_data.has_symptom = tot.has_symptom
WHERE max_utility_data.rank = 1;
                                         '''))

    for row in result:
        print(row)

except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

(False, False, 8.977617133408785, 0.6925, 6.2169998220769696)
(True, True, 5.759349569678307, 0.3075, 1.771000020138798)


The following code gives the overall maximum expected utility from solving the influence diagram.

In [31]:
try:
    with engine.begin() as connection:
        result = connection.execute(text('''
        WITH tot AS (
     SELECT has_symptom, SUM(prob) AS sum_prob FROM jpd GROUP BY has_symptom
),
prob_data AS 
(
    SELECT jpd.has_pathological_state, jpd.has_symptom, jpd.has_disease, 
           jpd.prob / tot.sum_prob as prob1,
           p.id as p_id, d.id as d_id
    FROM jpd
    JOIN tot ON jpd.has_symptom = tot.has_symptom
    JOIN d ON d.has_disease = jpd.has_disease
    JOIN p ON p.has_pathological_state = jpd.has_pathological_state
),
sum_util_data AS 
(
    SELECT prob_data.has_symptom, t.treat, SUM(prob_data.prob1 * u.utility) as sum_utility 
    FROM prob_data
    JOIN u ON (u.p_id = prob_data.p_id AND u.d_id = prob_data.d_id)
    JOIN t ON (t.id = u.t_id)
    GROUP BY prob_data.has_symptom, t.id
),
max_utility_data AS
(
    SELECT *,
        RANK() OVER (PARTITION BY has_symptom ORDER BY sum_utility DESC) as rank
    FROM sum_util_data
),
final_data AS
(
    SELECT max_utility_data.has_symptom, max_utility_data.treat, max_utility_data.sum_utility, tot.sum_prob, max_utility_data.sum_utility * tot.sum_prob as total_utility
    FROM max_utility_data
    JOIN tot ON max_utility_data.has_symptom = tot.has_symptom
    WHERE max_utility_data.rank = 1
)

SELECT SUM(total_utility) as sum_total_utility FROM final_data;
 ''')) 
        # Fetch the result and print it
    for row in result:
        print(row[0])  # print only the first element of the tuple
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

7.987999842215768
